# HW 4 - Building a normalized RDB
 
The goal of this homework is to take a semi-structured non-normalized CSV file and turn it into a set of normalized tables that you then push to your postgres database on AWS.  
 
The original dataset contains 100k district court decisions, although I've downsampled it to only 1000 rows to make the uploads faster.  Each row contains info about a judge, their demographics, party affiliation, etc.  Rows also contain information about the case they were deciding on.  Was it a criminal or civil case?  What year was it?  Was the direction of the decision liberal or conservative?  
 
While the current denormalized format is fine for analysis, it's not fine for a database as it violates many normalization rules. Your goal is to normalize it by designing a simple schema, then wrangling it into the proper dataframes, then pushing it all to AWS.  
 
For the first part of this assignment you should wind up with three tables.  One with case information, one with judge information, one that has casetype information, and the last with case category information.  Each table should be reduced so that there are not then repeating rows, and primary keys should be assigned within each.  These tables should be called 'cases', 'judges', and 'casetype'.
 
For the last part you should make a rollup table that calculates the percent of liberal decisions for each party level and each case category.  This will allow for one to get a quick look at how the political party affiliation of judges impacts the direction of a decision for different case categories (e.g. criminal, civil, labor). 

## Bring in data, explore, make schema

Start by bringing in your data to `cases`. Call a `.head()` on it to see what columns are there and what they contain.

In [ ]:
import pandas as pd
cases = pd.read_csv('https://ista322data.s3-us-west-1.amazonaws.com/cases_denorm_small.csv')

In [ ]:
# head of cases
cases.head()

,judge_name,party_name,gender_name,race_name,case_id,case_year,casetype_id,casetype_name,category_id,category_name,libcon_id,libcon_name
0,"Thompson, Myron H.",Democrat,male,African-American/black,28321332,2011,3,criminal court motions,1,Criminal Justice Cases,0,Conservative
1,"Shoob, Marvin A.",Democrat,male,white/caucasian,18110669,1993,14,free of religion,2,Civil Liberties/Rights Cases,1,Liberal
2,"Bua, Nicholas J.",Democrat,male,white/caucasian,15660871,1983,2,habeas corpus-state,1,Criminal Justice Cases,0,Conservative
3,"Kovachevich, Elizabeth",Republican,female,white/caucasian,17770934,1991,2,habeas corpus-state,1,Criminal Justice Cases,0,Conservative
4,"Gilliam, Earl B.",Independent/Other/Unknown,male,white/caucasian,26621195,2009,6,alien petitions,2,Civil Liberties/Rights Cases,0,Conservative


### Make schema
 
OK, given that head, you need to make three related tables that will make up a normalized database.  Those tables are 'cases', 'judges', and 'casetype'.  If it's not clear what info should go into each, explore the data more, ask on slack, etc.  
 
Remember, you might not have keys, will need to reduce the rows, select certain columns, etc.  There isn't a defined path here.  

Here's a textbox to define what should be in your schema... feel free to add to 

---

this. 

Schema of the 3 different tables.



## Make cases table - 3 points

Start by making a table that contains just each case's info.  I would call this table that you're going to upload `cases_df` so you don't overwrite your raw data.  

This table should have six columns and 1000 rows.  

Note, one of these columns should be a judge_id that links to the judges table. You'll need to make this foreign key.  

Also, you can leave 'category_name' in this table as well as its id.  Normally you'd split that off into it's own table as well, but you're already doing that for casetype which is enough for now.  

In [ ]:
# Make judge_id in cases
cases['judge_id'] = pd.factorize(cases['judge_name'])[0].astype(str)
cases

,judge_name,party_name,gender_name,race_name,case_id,case_year,casetype_id,casetype_name,category_id,category_name,libcon_id,libcon_name,judge_id
0,"Thompson, Myron H.",Democrat,male,African-American/black,28321332,2011,3,criminal court motions,1,Criminal Justice Cases,0,Conservative,0
1,"Shoob, Marvin A.",Democrat,male,white/caucasian,18110669,1993,14,free of religion,2,Civil Liberties/Rights Cases,1,Liberal,1
2,"Bua, Nicholas J.",Democrat,male,white/caucasian,15660871,1983,2,habeas corpus-state,1,Criminal Justice Cases,0,Conservative,2
3,"Kovachevich, Elizabeth",Republican,female,white/caucasian,17770934,1991,2,habeas corpus-state,1,Criminal Justice Cases,0,Conservative,3
4,"Gilliam, Earl B.",Independent/Other/Unknown,male,white/caucasian,26621195,2009,6,alien petitions,2,Civil Liberties/Rights Cases,0,Conservative,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,"Weiner, Charles R.",Democrat,male,white/caucasian,15360056,1981,17,employee v. employer,3,Labor & Economic Cases,0,Conservative,550
996,"Rovner, Illana D.",Republican,female,white/caucasian,17781450,1991,11,14th amendment,2,Civil Liberties/Rights Cases,0,Conservative,314
997,"Karas, Kenneth M.",Republican,female,white/caucasian,28120514,2011,26,handicapped rights,2,Civil Liberties/Rights Cases,0,Conservative,551
998,"White, George W.",Democrat,male,African-American/black,16930563,1988,10,racial discrimination,2,Civil Liberties/Rights Cases,0,Conservative,552


In [ ]:
# select necessary columns to make cases_df
cases_df = cases[['case_id','case_year','judge_id','casetype_id','casetype_name','category_name']]

In [ ]:
# What's the shape of cases_df?
cases_df.shape

(1000, 6)

### Make cases table in your database

Go bring over your connection function from the last workbook. **This must be in here for me to grade the homework** If I can't access your database you'll get a zero.  It's a good idea to bring our other exploratory functions as well.  

Once you do that you'll need to do the following

* Connect, make a table called 'cases' with the correct column names and data types.  Be sure to execute and commit the table.  
* Make tuples of your data
* Write a SQL string that allows you to insert each tuple of data into the correct columns
* Execute the string many times to fill out 'cases'
* Commit changes and check the table.  

I'm not going to leave a full roadmap beyond this.  Feel free to add cells as needed to do the above.  

In [ ]:
import psycopg2

In [ ]:
def get_conn_cur():
  # UPDATE WITH YOUR INFO!
  conn = psycopg2.connect(
  host="database-1.cvwfo3fm2djf.us-east-1.rds.amazonaws.com",
  database="hw4_db",
  user="postgres",
  password="password",
  port='5432')
  
  cur = conn.cursor()
  return(conn, cur)

In [ ]:
def run_query(query_string):

  conn, cur = get_conn_cur() # get connection and cursor

  cur.execute(query_string) # executing string as before

  my_data = cur.fetchall() # fetch query data as before

  # here we're extracting the 0th element for each item in cur.description
  colnames = [desc[0] for desc in cur.description]

  cur.close() # close
  conn.close() # close

  return(colnames, my_data) # return column names AND data

# Column name function for checking out what's in a table
def get_column_names(table_name): # arguement of table_name
  conn, cur = get_conn_cur() # get connection and cursor

  # Now select column names while inserting the table name into the WERE
  column_name_query =  """SELECT column_name FROM information_schema.columns
       WHERE table_name = '%s' """ %table_name

  cur.execute(column_name_query) # exectue
  my_data = cur.fetchall() # store

  cur.close() # close
  conn.close() # close

  return(my_data) # return

# Check table_names
def get_table_names():
  conn, cur = get_conn_cur() # get connection and cursor

  # query to get table names
  table_name_query = """SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public' """

  cur.execute(table_name_query) # execute
  my_data = cur.fetchall() # fetch results

  cur.close() #close cursor
  conn.close() # close connection

  return(my_data) # return your fetched results
  
# make sql_head function
def sql_head(table_name):
  conn, cur = get_conn_cur() # get connection and cursor

  # Now select column names while inserting the table name into the WERE
  head_query =  """SELECT * FROM %s LIMIT 5; """ %table_name

  cur.execute(head_query) # exectue
  colnames = [desc[0] for desc in cur.description] # get column names
  my_data = cur.fetchall() # store first five rows

  cur.close() # close
  conn.close() # close

  df = pd.DataFrame(data = my_data, columns = colnames) # make into df

  return(df) # return

In [ ]:
tq = """CREATE TABLE cases (
          case_id INTEGER PRIMARY KEY,
          case_year INTEGER NOT NULL , 
          judge_id INTEGER NOT NULL,
          casetype_id INTEGER NOT NULL,
          casetype_name VARCHAR(300) NOT NULL,
          category_name VARCHAR(300) NOT NULL
          );"""

In [ ]:
conn, cur = get_conn_cur()

In [ ]:
cur.execute(tq)

In [ ]:
conn.commit()

In [ ]:
get_table_names()

[('weekly_stats',), ('cases',)]

In [ ]:
get_column_names(table_name='cases')

[('case_id',),
 ('case_year',),
 ('judge_id',),
 ('casetype_id',),
 ('casetype_name',),
 ('category_name',)]

In [ ]:
import numpy as np
data_tups = [tuple(data) for data in cases_df.to_numpy()]

In [ ]:
insert_query = """ INSERT INTO cases(case_id,case_year,judge_id,casetype_id,casetype_name,category_name) VALUES(%s, %s, %s, %s, %s,%s); """ 

In [ ]:
conn, cur = get_conn_cur()
cur.executemany(insert_query, data_tups)
conn.commit()

In [ ]:
# Use sql_head to check cases
sql_head(table_name='cases')

,case_id,case_year,judge_id,casetype_id,casetype_name,category_name
0,28321332,2011,0,3,criminal court motions,Criminal Justice Cases
1,18110669,1993,1,14,free of religion,Civil Liberties/Rights Cases
2,15660871,1983,2,2,habeas corpus-state,Criminal Justice Cases
3,17770934,1991,3,2,habeas corpus-state,Criminal Justice Cases
4,26621195,2009,4,6,alien petitions,Civil Liberties/Rights Cases


## Make judges - 2 points

Now make your  judges table from the original `cases` dataframe (not the SQL table you just made).  

Judges should have five columns, including the `judge_id` column you made. There should be 553 rows after you drop duplicates (remember that judges may have had more than one case).  

After you  make the dataset go and push to a SQL table called 'judges'. 

In [ ]:
judges_df = cases[['judge_id','judge_name','party_name','gender_name','race_name']].drop_duplicates()

In [ ]:
judges_df.shape

(553, 5)

In [ ]:
new_q = """CREATE TABLE judges (
          judge_id INTEGER PRIMARY KEY,
          judge_name VARCHAR(300) NOT NULL,
          party_name VARCHAR(300) NOT NULL,
          gender_name VARCHAR(300) NOT NULL,
          race_name VARCHAR(300) NOT NULL
          );"""

In [ ]:
cur.execute(new_q)

In [ ]:
conn.commit()

In [ ]:
get_table_names()

[('weekly_stats',), ('cases',), ('judges',)]

In [ ]:
get_column_names(table_name="judges")

[('judge_id',),
 ('judge_name',),
 ('party_name',),
 ('gender_name',),
 ('race_name',)]

In [ ]:
data_tups2 = [tuple(data) for data in judges_df.to_numpy()]

In [ ]:
insert_judges_query = """ INSERT INTO judges(judge_id,judge_name,party_name,gender_name,race_name) VALUES(%s, %s, %s, %s, %s); """ 

In [ ]:
conn, cur = get_conn_cur()
cur.executemany(insert_judges_query, data_tups2)
conn.commit()

In [ ]:
sql_head(table_name='judges')

,judge_id,judge_name,party_name,gender_name,race_name
0,0,"Thompson, Myron H.",Democrat,male,African-American/black
1,1,"Shoob, Marvin A.",Democrat,male,white/caucasian
2,2,"Bua, Nicholas J.",Democrat,male,white/caucasian
3,3,"Kovachevich, Elizabeth",Republican,female,white/caucasian
4,4,"Gilliam, Earl B.",Independent/Other/Unknown,male,white/caucasian


## Make casetype - 2 points

Go make the casetype table.  This should have only two columns that allow you to link the casetype name back to the ID in the 'cases' table. There should be 27 rows as well. 

In [ ]:
casetype_df = cases[['casetype_id','casetype_name']].drop_duplicates()

In [ ]:
casetype_df.shape

(27, 2)

In [ ]:
casetype_q = """CREATE TABLE casetypes (
          casetype_id INTEGER PRIMARY KEY,
          casetype_name VARCHAR(300) NOT NULL
          );"""

In [ ]:
cur.execute(casetype_q)

In [ ]:
conn.commit()

In [ ]:
get_table_names()

[('weekly_stats',), ('cases',), ('judges',), ('casetypes',)]

In [ ]:
get_column_names(table_name="casetypes")

[('casetype_id',), ('casetype_name',)]

In [ ]:
data_tups3 = [tuple(data) for data in casetype_df.to_numpy()]

In [ ]:
insert_casetypes_query = """ INSERT INTO casetypes(casetype_id,casetype_name) VALUES(%s, %s); """ 

In [ ]:
conn, cur = get_conn_cur()
cur.executemany(insert_casetypes_query, data_tups3)
conn.commit()

In [ ]:
sql_head(table_name="casetypes")

,casetype_id,casetype_name
0,3,criminal court motions
1,14,free of religion
2,2,habeas corpus-state
3,6,alien petitions
4,19,environmental protection


## A quick test of your tables

Below is a query to get the number of unique judges that have ruled on criminal court motion cases.  You should get a value of 119 as your return if your database is set up correctly!

In [ ]:
run_query("""SELECT COUNT(DISTINCT(judges.judge_id)) FROM cases 
                JOIN judges ON cases.judge_id = judges.judge_id 
                WHERE casetype_id = (SELECT casetype_id FROM casetypes
                                        WHERE casetype_name = 'criminal court motions'); """)
              

(['count'], [(119,)])

## Make rollup table - 3 points
 
Now let's make that rollup table!  The goal here is to make a summary table easily accessed.  We're going to roll the whole thing up by the judges party and the category, but you could imagine doing this for each judge to track how they make decisions over time which would then be useful for an analytics database.  The one we're making could also be used as a dimension table where we needed overall party averages.  
 
We want to get a percentage of liberal decisions by each grouping level (party_name, category_name). To do this we need first, the number of cases seen at each level, and second, the number of liberal decisions made at each level.  `cases` contains the columns `libcon_id` which is a 0 if the decision was conservative in its ruling, and a 1 if it was liberal in its ruling.  Thus, you can get a percentage of liberal decisions if you divide the sum of that column by the total observations.  Your `agg()` can both get the sum and count.  
 
After you groupby you'll need to reset the index, rename the columns, then make the percentage.  
 
Once you do that you can push to a SQL table called 'rollup'
 
Let's get started

In [ ]:
# Make a groupby called cases_rollup.  This should group by party_name and categrory name.  It should aggregate the count and sum of libcon_id
cases_rollup = cases.groupby([cases['party_name'], cases['category_name']]).agg({ 'libcon_id':['count','sum']})

In [ ]:
cases_rollup

libcon_id     
                                                           count  sum
party_name                category_name                              
Democrat                  Civil Liberties/Rights Cases       218  112
                          Criminal Justice Cases             107   36
                          Labor & Economic Cases             126   69
Independent/Other/Unknown Civil Liberties/Rights Cases        12    5
                          Criminal Justice Cases               8    5
                          Labor & Economic Cases              13    6
Republican                Civil Liberties/Rights Cases       237   73
                          Criminal Justice Cases             125   34
                          Labor & Economic Cases             154   67

In [ ]:
# reset your index
cases_rollup.columns = cases_rollup.columns.map('_'.join)
cases_rollup = cases_rollup.reset_index()

In [ ]:
cases_rollup

,party_name,category_name,libcon_id_count,libcon_id_sum
0,Democrat,Civil Liberties/Rights Cases,218,112
1,Democrat,Criminal Justice Cases,107,36
2,Democrat,Labor & Economic Cases,126,69
3,Independent/Other/Unknown,Civil Liberties/Rights Cases,12,5
4,Independent/Other/Unknown,Criminal Justice Cases,8,5
5,Independent/Other/Unknown,Labor & Economic Cases,13,6
6,Republican,Civil Liberties/Rights Cases,237,73
7,Republican,Criminal Justice Cases,125,34
8,Republican,Labor & Economic Cases,154,67


In [ ]:
# rename your columns now.  Keep the first to the same but call the last two 'total_cases' and 'num_lib_decisions'
cases_rollup.columns = ['party_name','category_name','total_cases','num_lib_decisions']
cases_rollup

,party_name,category_name,total_cases,num_lib_decisions
0,Democrat,Civil Liberties/Rights Cases,218,112
1,Democrat,Criminal Justice Cases,107,36
2,Democrat,Labor & Economic Cases,126,69
3,Independent/Other/Unknown,Civil Liberties/Rights Cases,12,5
4,Independent/Other/Unknown,Criminal Justice Cases,8,5
5,Independent/Other/Unknown,Labor & Economic Cases,13,6
6,Republican,Civil Liberties/Rights Cases,237,73
7,Republican,Criminal Justice Cases,125,34
8,Republican,Labor & Economic Cases,154,67


Now make a new column called 'percent_liberal'

This should calucalte the percentage of decisions that were liberal in nature.  Multiple it by 100 so that it's a full percent.  Also use the `round()` function on the whole thing to keep it in whole percentages.

In [ ]:
# make your metric called 'percent_liberal' 
cases_rollup['percent_liberal'] = round((cases_rollup['num_lib_decisions'] / cases_rollup['total_cases']) * 100)
cases_rollup

,party_name,category_name,total_cases,num_lib_decisions,percent_liberal
0,Democrat,Civil Liberties/Rights Cases,218,112,51.0
1,Democrat,Criminal Justice Cases,107,36,34.0
2,Democrat,Labor & Economic Cases,126,69,55.0
3,Independent/Other/Unknown,Civil Liberties/Rights Cases,12,5,42.0
4,Independent/Other/Unknown,Criminal Justice Cases,8,5,62.0
5,Independent/Other/Unknown,Labor & Economic Cases,13,6,46.0
6,Republican,Civil Liberties/Rights Cases,237,73,31.0
7,Republican,Criminal Justice Cases,125,34,27.0
8,Republican,Labor & Economic Cases,154,67,44.0


Now go and push the whole thing to a table called 'rollup'

There should be five columns and nine rows. 

In [ ]:
rollup_q = """CREATE TABLE  rollup(
          party_name VARCHAR(300) NOT NULL,
          category_name VARCHAR(300) NOT NULL,
          total_cases INTEGER NOT NULL,
          num_lib_decisions INTEGER NOT NULL,
          percent_liberal NUMERIC PRIMARY KEY
          );"""

In [ ]:
cur.execute(rollup_q)

In [ ]:
conn.commit()

In [ ]:
get_table_names()

[('weekly_stats',), ('cases',), ('judges',), ('casetypes',), ('rollup',)]

In [ ]:
get_column_names(table_name='rollup')

[('party_name',),
 ('category_name',),
 ('total_cases',),
 ('num_lib_decisions',),
 ('percent_liberal',)]

In [ ]:
data_tups4 = [tuple(data) for data in cases_rollup.to_numpy()]

In [ ]:
insert_cases_rollup_query = """ INSERT INTO rollup(party_name,category_name,total_cases,num_lib_decisions,percent_liberal) VALUES(%s, %s,%s,%s,%s); """ 

In [ ]:
conn, cur = get_conn_cur()
cur.executemany(insert_cases_rollup_query, data_tups4)
conn.commit()

In [ ]:
# Check
sql_head('rollup')

,party_name,category_name,total_cases,num_lib_decisions,percent_liberal
0,Democrat,Civil Liberties/Rights Cases,218,112,51.0
1,Democrat,Criminal Justice Cases,107,36,34.0
2,Democrat,Labor & Economic Cases,126,69,55.0
3,Independent/Other/Unknown,Civil Liberties/Rights Cases,12,5,42.0
4,Independent/Other/Unknown,Criminal Justice Cases,8,5,62.0
